We are testing the functionality of pyzx_to_cirq by applying to see if 

circuit == pyzx_to_cirq(circuit_from_cirq(circuit))

In [131]:
import cirq
import pyzx as zx
from fractions import Fraction
import numpy as np

In [132]:
MAX_QUBITS = 8

def circuit_from_cirq(circuit: cirq.Circuit) -> zx.Circuit:
# Verified and debugged. See code_verification\circuit_from_cirq.ipynb
    '''
    This function converts a Cirq circuit to a PyZX circuit.
    It supports the following gates:
    - H (Hadamard)
    - X, Z (with phase)
    - CNOT (controlled-NOT)
    '''
    zx_circ = zx.Circuit(MAX_QUBITS)
    for i, moment in enumerate(circuit):
        for op in moment:
            gate = op.gate
            q = [q.x for q in op.qubits]
            if isinstance(gate, cirq.HPowGate) and np.isclose(gate.exponent, 1):
                zx_circ.add_gate("H", q[0])
            elif isinstance(gate, cirq.XPowGate):
                phase = Fraction(gate.exponent).limit_denominator(1000)
                zx_circ.add_gate("XPhase", q[0], phase=phase)
            elif isinstance(gate, cirq.ZPowGate):
                phase = Fraction(gate.exponent).limit_denominator(1000)
                zx_circ.add_gate("ZPhase", q[0], phase=phase)
            elif isinstance(gate, cirq.YPowGate):
                pass
                # We avoid using YPhase
                # phase = Fraction(gate.exponent).limit_denominator(1000)
                # zx_circ.add_gate("YPhase", q[0], phase=phase)
                # displays YPhase gate via combination of X and Z phases
            elif isinstance(gate, cirq.CNotPowGate) and np.isclose(gate.exponent, 1):
                zx_circ.add_gate("CNOT", q[0], q[1])
            elif isinstance(gate, cirq.CZPowGate) and np.isclose(gate.exponent, 1):
                zx_circ.add_gate("CZ", q[0], q[1])
            else:
                print(f"    -> Unsupported or unknown gate: {type(gate)}")
    return zx_circ

In [133]:
def pyzx_to_cirq(pyzx_circuit):
# Verified and debugged. See code_verification\pyzx_to_cirq.ipynb
    '''
    Converts a PyZX circuit to a Cirq circuit.
    Supports the following gates:
    - XPhase (X with phase)
    - ZPhase (Z with phase)
    - Hadamard (H)
    - CNOT (controlled-NOT)
    - CZ (controlled-Z)
    '''
    pi = 3.141592653589793
    ops = []
    def get_qubits_used():
        max_index = 0
        for gate in pyzx_circuit.gates:
            targets = []
            if hasattr(gate, 'target'):
                t = gate.target
                if isinstance(t, (list, tuple)):
                    targets.extend(t)
                else:
                    targets.append(t)
            if hasattr(gate, 'control'):
                targets.append(gate.control)
            if targets:
                max_index = max(max_index, max(targets))
        return max_index + 1
    num_qubits = pyzx_circuit.n_qubits if hasattr(pyzx_circuit, 'n_qubits') else get_qubits_used()
    cirq_qubits = [cirq.LineQubit(i) for i in range(num_qubits)]
    for gate in pyzx_circuit.gates:
        name = gate.name.upper()
        if name == "ZPHASE":
            q = gate.target if isinstance(gate.target, int) else gate.target[0]
            ops.append(cirq.rz(gate.phase * pi).on(cirq_qubits[q]))
        elif name == "XPHASE":
            q = gate.target if isinstance(gate.target, int) else gate.target[0]
            ops.append(cirq.rx(gate.phase * pi).on(cirq_qubits[q]))
        elif name == "HAD":
            q = gate.target if isinstance(gate.target, int) else gate.target[0]
            ops.append(cirq.H(cirq_qubits[q]))
        elif name == "CNOT":
            ctrl = gate.control
            tgt = gate.target if isinstance(gate.target, int) else gate.target[0]
            ops.append(cirq.CNOT(cirq_qubits[ctrl], cirq_qubits[tgt]))
        elif name == "CZ":
            ctrl = gate.control
            tgt = gate.target if isinstance(gate.target, int) else gate.target[0]
            ops.append(cirq.CZ(cirq_qubits[ctrl], cirq_qubits[tgt]))
        else:
            pass  # or: print(f"Unrecognized gate {name}, skipping.")
    return cirq.Circuit(ops)


In [134]:
import numpy as np
def generate_random_superconducting_circuit(n_qubits, depth):
# Verified 
    qubits = [cirq.LineQubit(i) for i in range(n_qubits)]
    circuit = cirq.Circuit()
    for _ in range(depth):
        layer = []
        i = 0
        for i in range(n_qubits):
            op_type = np.random.choice(['rx', 'rz', 'cx'])
            if op_type == 'rx':
                theta = np.random.choice([(-7*np.pi/4), (-3*np.pi/2), (-5*np.pi/4), (-np.pi), (-3*np.pi/4), (-np.pi/2), (-np.pi/4), 0,
                                        (7*np.pi/4), (3*np.pi/2), (5*np.pi/4), (np.pi), (3*np.pi/4), (np.pi/2), (np.pi/4)])
                layer.append(cirq.rx(theta)(qubits[i]))
            elif op_type == 'rz':
                phi = np.random.choice([(-7*np.pi/4), (-3*np.pi/2), (-5*np.pi/4), (-np.pi), (-3*np.pi/4), (-np.pi/2), (-np.pi/4), 0,
                                        (7*np.pi/4), (3*np.pi/2), (5*np.pi/4), (np.pi), (3*np.pi/4), (np.pi/2), (np.pi/4)])
                layer.append(cirq.rz(phi)(qubits[i]))
            elif op_type == 'cx' and i < len(qubits) - 1:
                if np.random.rand() < 0.5:
                    layer.append(cirq.CNOT(qubits[i], qubits[i + 1]))
        circuit.append(layer)
    return circuit

In [135]:
import numpy as np
def circuits_equivalent(circ1: cirq.Circuit, circ2: cirq.Circuit):
    U1 = cirq.unitary(circ1)
    U2 = cirq.unitary(circ2)
    diff = U1.conj().T @ U2
    phases = np.angle(np.linalg.eigvals(diff))
    global_phase = np.exp(1j * phases[0])
    return np.allclose(global_phase * U1, U2, atol=1E-8)

In [136]:
circuit = generate_random_superconducting_circuit(MAX_QUBITS,50)

print("THE CIRCUIT:")
print(circuit)

zx_circuit = circuit_from_cirq(circuit)
print("THE ZX CIRCUIT FROM circuit_from_cirq(circuit):")
zx.draw(zx_circuit)

cirq_circuit = pyzx_to_cirq(zx_circuit)
print("THE CIRCUIT FROM pyzx_to_cirq(zx_circuit):")
print(cirq_circuit)

print(f"Eqivalent circuits?: {circuits_equivalent(circuit, cirq_circuit)}")

THE CIRCUIT:
0: ───Rz(-1.75π)───Rz(0.75π)───Rx(-1.25π)───Rz(-1.75π)───Rz(-π)───Rz(π)────────Rx(1.5π)─────Rx(0.25π)──────────────────────────────@────────────Rx(-π)────────────────────@──────────Rx(1.25π)───Rz(0.25π)───Rx(-0.5π)────Rz(1.75π)───Rx(0.25π)───────────────────────────────────────────@───────────────────────@────────────────────────@────────────Rx(-0.25π)───Rz(-1.75π)───Rz(1.5π)─────Rz(-0.5π)─────────────────@───────────────────────────────────@────────────Rz(1.75π)───@────────────Rz(0.75π)────@────────────Rz(1.25π)───Rz(0.75π)─────────────────@────────────Rx(π)─────────────────────@───────────Rz(0.25π)────Rz(-0.5π)────Rz(1.25π)────@─────────────────────────@────────────Rx(1.75π)────Rx(1.25π)─────────────────@─────────────────────────────────────
                                                                                                                                   │                                      │                                                              

THE CIRCUIT FROM pyzx_to_cirq(zx_circuit):
0: ───Rz(-1.75π)───Rz(0.75π)───Rx(-1.25π)───Rz(-1.75π)───Rz(-π)───Rz(π)────────Rx(1.5π)─────Rx(0.25π)──────────────────────────────@────────────Rx(-π)────────────────────@──────────Rx(1.25π)───Rz(0.25π)───Rx(-0.5π)────Rz(1.75π)───Rx(0.25π)───────────────────────────────────────────@───────────────────────@────────────────────────@────────────Rx(-0.25π)───Rz(-1.75π)───Rz(1.5π)─────Rz(-0.5π)─────────────────@───────────────────────────────────@────────────Rz(1.75π)───@────────────Rz(0.75π)────@────────────Rz(1.25π)───Rz(0.75π)─────────────────@────────────Rx(π)─────────────────────@───────────Rz(0.25π)────Rz(-0.5π)────Rz(1.25π)────@─────────────────────────@────────────Rx(1.75π)────Rx(1.25π)─────────────────@─────────────────────────────────────
                                                                                                                                   │                                      │                                